In [6]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
# Use Pandas to load dataset from csv file
import pandas as pd

# Make sure we don't get any GPU errors
physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [7]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [8]:
# Check the shape of the training and test data
print("Train Images Shape:", train_images.shape)
print("Test Images Shape:", test_images.shape)
print("Train Labels Shape:", train_labels.shape)
print("Test Labels Shape:", test_labels.shape)

Train Images Shape: (60000, 28, 28)
Test Images Shape: (10000, 28, 28)
Train Labels Shape: (60000,)
Test Labels Shape: (10000,)


In [9]:
train_images = train_images.reshape(-1,28,28,1).astype("float32") / 255.0
test_images = test_images.reshape(-1,28,28,1).astype("float32") / 255.0

In [10]:
train_images.shape

(60000, 28, 28, 1)

In [21]:
# CNN -> BatchNorm -> ReLU (common structure)
# x10 (a lot of code to write!)

class CNNBLOCK (layers.Layer):
  def __init__(self, out_chanels, kernel_size = 3) :
    super(CNNBLOCK, self).__init__()
    self.conv = layers.Conv2D (out_chanels, kernel_size, padding = 'same')
    self.bn = layers.BatchNormalization()

  def call (self, input_tensor, training = False):
    x = self.conv(input_tensor)
    x = self.bn(x, training = training)
    x = tf.nn.relu(x)
    return x

In [22]:
model = keras.Sequential(
    [
        CNNBLOCK(32),
        CNNBLOCK(64),
        CNNBLOCK(128),
        layers.Flatten(),
        layers.Dense(10),
    ]
)

In [23]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(),
    metrics=["accuracy"],
)

In [24]:
model.fit (train_images, train_labels, batch_size=64, epochs=5, verbose =2 )

Epoch 1/5
938/938 - 17s - loss: 0.5716 - accuracy: 0.9466 - 17s/epoch - 18ms/step
Epoch 2/5
938/938 - 11s - loss: 0.0934 - accuracy: 0.9812 - 11s/epoch - 11ms/step
Epoch 3/5
938/938 - 11s - loss: 0.0338 - accuracy: 0.9902 - 11s/epoch - 11ms/step
Epoch 4/5
938/938 - 11s - loss: 0.0266 - accuracy: 0.9918 - 11s/epoch - 11ms/step
Epoch 5/5
938/938 - 11s - loss: 0.0274 - accuracy: 0.9912 - 11s/epoch - 11ms/step


In [25]:
model.evaluate(test_images, test_labels, batch_size=64, verbose =2 )

157/157 - 1s - loss: 0.0534 - accuracy: 0.9837 - 1s/epoch - 7ms/step


[0.05342560261487961, 0.9836999773979187]